# El siguiente código contiene:

### 1. Pruebas para evaluación de diversos modelos LLM de Hugging Face
### 2. Resultados de cada modelo
### Fecha:2024


In [23]:
from transformers import AutoTokenizer, RobertaForTokenClassification
import torch
from transformers import AutoTokenizer
from json import JSONEncoder
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [24]:
text="""So, if you're a NASA scientist, you should be able to tell me the whole story about the Face On Mars, which obviously is evidence that there is life on Mars, and that the face was created by aliens, correct?" No, twenty five years ago, our Viking 1 spacecraft was circling the planet, snapping photos, when it spotted the shadowy likeness of a human face. Us scientists figured out that it was just another Martian mesa, common around Cydonia, only this one had shadows that made it look like an Egyption Pharaoh. Very few days later, we revealed the image for all to see, and we made sure to note that it was a huge rock formation that just resembled a human head and face, but all of it was formed by shadows. We only announced it because we thought it would be a good way to engage the public with NASA's findings, and atrract attention to Mars-- and it did.

The face on Mars soon became a pop icon; shot in movies, appeared in books, magazines, radio talk shows, and haunted grocery store checkout lines for 25 years. Some people thought the natural landform was evidence of life on Mars, and that us scientists wanted to hide it, but really, the defenders of the NASA budget wish there was ancient civilization on Mars. We decided to take another shot just to make sure we weren't wrong, on April 5, 1998. Michael Malin and his Mars Orbiter camera team took a picture that was ten times sharper than the original Viking photos, revealing a natural landform, which meant no alien monument. "But that picture wasn't very clear at all, which could mean alien markings were hidden by haze" Well no, yes that rumor started, but to prove them wrong on April 8, 2001 we decided to take another picture, making sure it was a cloudless summer day. Malin's team captured an amazing photo using the camera's absolute maximum revolution. With this camera you can discern things in a digital image, 3 times bigger than the pixel size which means if there were any signs of life, you could easily see what they were. What the picture showed was the butte or mesa, which are landforms common around the American West."""



## Procesar json de salidas de modelos

In [3]:
import pandas as pd
def obtener_dataframe(data):
        
       
   data_flattened = [flatten_json(class_info) for class_info in data]
   df = pd.DataFrame(data_flattened)
            
   return df
    ###
    ### funcion "flatten_json" tomada de https://levelup.gitconnected.com/a-deep-dive-into-nested-json-to-data-frame-with-python-69bdabb41938 
    ### Renu Khandelwal Jul 23, 2023
def flatten_json(y):
        try:
            out = {}
    
            def flatten(x, name=''):
                if type(x) is dict:
                    for a in x:
                        flatten(x[a], name + a + '_')
                elif type(x) is list:
                    i = 0
                    for a in x:
                        flatten(a, name + str(i) + '_')
                        i += 1
                else:
                    out[name[:-1]] = x
    
            flatten(y)
            return out
        except json.JSONDecodeError:
            print("Error: The JSON document could not be decoded.")
        except TypeError:
            print("Error: Invalid operation or function argument type.")
        except KeyError:
            print("Error: One or more keys do not exist.")
        except ValueError:
            print("Error: Invalid value detected.")
        except Exception as e:
            # Catch any other exceptions
            print(f"An unexpected error occurred: {str(e)}") 



## 1 FacebookAI/xlm-roberta-large-finetuned-conll03-english

In [28]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large-finetuned-conll03-english")
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])
model_args={"trust_remote_code": True}
model = AutoModelForTokenClassification.from_pretrained("FacebookAI/xlm-roberta-large-finetuned-conll03-english", **model_args)
with torch.no_grad():
    logits = model(input_ids).logits
predicted_token_class_ids = logits.argmax(-1)

predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

labels = predicted_token_class_ids
loss = model(input_ids, labels=labels).loss
print('loss:',round(loss.item(), 2))
model.config.id2label


Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


loss: 0.0


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{0: 'LABEL_0', 1: 'LABEL_1'}

In [29]:
model = AutoModelForTokenClassification.from_pretrained("FacebookAI/xlm-roberta-large", **model_args)
model.config.id2label

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{0: 'LABEL_0', 1: 'LABEL_1'}

## Métricas FacebookAI/xlm-roberta-large-finetuned-conll03-english

In [46]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("FacebookAI/xlm-roberta-large-finetuned-conll03-english")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)



Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-ORG', 'score': 0.9999913, 'index': 8, 'word': '▁NASA', 'start': 16, 'end': 20}, {'entity': 'I-MISC', 'score': 0.9999641, 'index': 23, 'word': '▁Face', 'start': 88, 'end': 92}, {'entity': 'I-MISC', 'score': 0.99989665, 'index': 24, 'word': '▁On', 'start': 93, 'end': 95}, {'entity': 'I-MISC', 'score': 0.97350365, 'index': 25, 'word': '▁Mars', 'start': 96, 'end': 100}, {'entity': 'I-LOC', 'score': 0.9999362, 'index': 36, 'word': '▁Mars', 'start': 152, 'end': 156}, {'entity': 'I-MISC', 'score': 0.9992086, 'index': 58, 'word': '▁Viking', 'start': 240, 'end': 246}, {'entity': 'I-MISC', 'score': 0.9989502, 'index': 59, 'word': '▁1', 'start': 247, 'end': 248}, {'entity': 'I-MISC', 'score': 0.999977, 'index': 97, 'word': '▁Marti', 'start': 407, 'end': 412}, {'entity': 'I-MISC', 'score': 0.99619055, 'index': 98, 'word': 'an', 'start': 412, 'end': 414}, {'entity': 'I-LOC', 'score': 0.9999354, 'index': 103, 'word': '▁Cy', 'start': 435, 'end': 437}, {'entity': 'I-LOC', 'score': 0.999

In [47]:

# Create DataFrame from flattened JSON
with open("metricas_Facebook.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
I-LOC      9
I-MISC    15
I-ORG      3
I-PER      3
dtype: int64


entity  word     
I-LOC   do           1
        nia          1
        ▁Cy          1
        ▁Mars        5
        ▁West        1
I-MISC  an           1
        bit          1
        er           1
        ion          1
        ▁1           1
        ▁American    1
        ▁Egypt       1
        ▁Face        1
        ▁Mars        2
        ▁Marti       1
        ▁On          1
        ▁Or          1
        ▁Viking      2
I-ORG   ▁NASA        3
I-PER   ▁Malin       2
        ▁Michael     1
dtype: int64

In [29]:
#pip install seqeval
from datasets import load_dataset, load_metric
metric = load_metric("seqeval",**model_args)
metric.compute(predictions=[predicted_tokens_classes], references=[predicted_tokens_classes])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 7},
 'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 7},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

## 2 manu/lilt-infoxlm-base

In [52]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("manu/lilt-infoxlm-base")

ValueError: The checkpoint you are trying to load has model type `liltrobertalike` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 3 projecte-aina/DEBERTA_CIEL

In [54]:

from transformers import pipeline

pipe = pipeline("ner", model="projecte-aina/DEBERTA_CIEL")
ner_entity_results = pipe(text, aggregation_strategy="simple")
print(ner_entity_results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'GPE', 'score': 0.6340015, 'word': 'Cy', 'start': 434, 'end': 437}, {'entity_group': 'location-other', 'score': 0.20539406, 'word': 'donia', 'start': 437, 'end': 442}, {'entity_group': 'organization-other', 'score': 0.3824667, 'word': 'NASA', 'start': 1168, 'end': 1173}, {'entity_group': 'person-other', 'score': 0.62630403, 'word': 'Michael Malin', 'start': 1311, 'end': 1325}, {'entity_group': 'organization-other', 'score': 0.28832194, 'word': 'Mars', 'start': 1333, 'end': 1338}, {'entity_group': 'organization-privatecompany', 'score': 0.13149069, 'word': 'Orbiter', 'start': 1338, 'end': 1346}, {'entity_group': 'person-other', 'score': 0.27721033, 'word': 'Malin', 'start': 1744, 'end': 1750}, {'entity_group': 'location-other', 'score': 0.44634995, 'word': 'American', 'start': 2093, 'end': 2102}, {'entity_group': 'location-other', 'score': 0.18410492, 'word': 'West', 'start': 2102, 'end': 2107}]


In [56]:
# Create DataFrame from flattened JSON
with open("3 metricas projecte-ainaDEBERTA_CIEL.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity_group']).size())
aux.groupby(['entity_group', 'word']).size()

entity_group
GPE                            1
location-other                 3
organization-other             2
organization-privatecompany    1
person-other                   2
dtype: int64


entity_group                 word         
GPE                          Cy               1
location-other               American         1
                             West             1
                             donia            1
organization-other           Mars             1
                             NASA             1
organization-privatecompany  Orbiter          1
person-other                 Malin            1
                             Michael Malin    1
dtype: int64

## 4 gunghio/distilbert-base-multilingual-cased-finetuned-conll2003-ner

In [58]:
from transformers import pipeline

pipe = pipeline("token-classification", model="gunghio/distilbert-base-multilingual-cased-finetuned-conll2003-ner")
#pipe = pipeline("ner", model="projecte-aina/DEBERTA_CIEL")
ner_entity_results = pipe(text, aggregation_strategy="simple")
print(ner_entity_results)

[{'entity_group': 'ORG', 'score': 0.98227155, 'word': 'NASA', 'start': 16, 'end': 20}, {'entity_group': 'MISC', 'score': 0.7845409, 'word': 'Face On Mars', 'start': 88, 'end': 100}, {'entity_group': 'LOC', 'score': 0.97792244, 'word': 'Mars', 'start': 152, 'end': 156}, {'entity_group': 'MISC', 'score': 0.9694425, 'word': 'Viking 1', 'start': 240, 'end': 248}, {'entity_group': 'MISC', 'score': 0.95397955, 'word': 'Martian', 'start': 407, 'end': 414}, {'entity_group': 'LOC', 'score': 0.92743087, 'word': 'Cydonia', 'start': 435, 'end': 442}, {'entity_group': 'MISC', 'score': 0.6778414, 'word': 'Egyption Ph', 'start': 496, 'end': 507}, {'entity_group': 'MISC', 'score': 0.6644676, 'word': '##oh', 'start': 510, 'end': 512}, {'entity_group': 'ORG', 'score': 0.9827271, 'word': 'NASA', 'start': 801, 'end': 805}, {'entity_group': 'LOC', 'score': 0.9836016, 'word': 'Mars', 'start': 843, 'end': 847}, {'entity_group': 'LOC', 'score': 0.97961295, 'word': 'Mars', 'start': 875, 'end': 879}, {'entity_g

In [60]:

with open("4 metricas gunghio distilbert-base-multilingual-cased-finetuned-conll2003-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity_group']).size())
aux.groupby(['entity_group', 'word']).size()

entity_group
LOC     7
MISC    6
ORG     4
PER     2
dtype: int64


entity_group  word         
LOC           American West    1
              Cydonia          1
              Mars             5
MISC          ##oh             1
              Egyption Ph      1
              Face On Mars     1
              Martian          1
              Viking           1
              Viking 1         1
ORG           Mars Orbiter     1
              NASA             3
PER           Malin            1
              Michael Malin    1
dtype: int64

## 5 mrm8488/distilbert-base-multi-cased-finetuned-typo-detection

In [62]:
from transformers import pipeline

typo_checker = pipeline(
    "ner",
    model="mrm8488/distilbert-base-multi-cased-finetuned-typo-detection",
    tokenizer="mrm8488/distilbert-base-multi-cased-finetuned-typo-detection"
)

result = typo_checker(text)
result[1:-1]


[{'entity': 'ok',
  'score': 0.9418772,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'ok',
  'score': 0.9061194,
  'index': 3,
  'word': 'if',
  'start': 4,
  'end': 6},
 {'entity': 'ok',
  'score': 0.9935272,
  'index': 4,
  'word': 'you',
  'start': 7,
  'end': 10},
 {'entity': 'ok',
  'score': 0.9791702,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'ok',
  'score': 0.99081236,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'ok',
  'score': 0.979658,
  'index': 7,
  'word': 'a',
  'start': 14,
  'end': 15},
 {'entity': 'ok',
  'score': 0.7008045,
  'index': 8,
  'word': 'NASA',
  'start': 16,
  'end': 20},
 {'entity': 'ok',
  'score': 0.9583886,
  'index': 9,
  'word': 'scientist',
  'start': 21,
  'end': 30},
 {'entity': 'ok',
  'score': 0.9387937,
  'index': 10,
  'word': ',',
  'start': 30,
  'end': 31},
 {'entity': 'typo',
  'score': 0.955443,
  'index': 11,
  'word': 'you',
  'start': 32,
  'end': 35},
 {'en

In [63]:
with open("5 metricas mrm8488.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
ok      328
typo    164
dtype: int64


entity  word   
ok                 7
        ##adow     3
        ##ara      1
        ##biter    1
        ##con      1
                  ..
typo    when       1
        which      4
        wrong      1
        ye         1
        you        3
Length: 298, dtype: int64

## 6 sagorsarker/codeswitch-spaeng-ner-lince 

In [128]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("sagorsarker/codeswitch-spaeng-ner-lince")

model = AutoModelForTokenClassification.from_pretrained("sagorsarker/codeswitch-spaeng-ner-lince")

ner_model = pipeline('ner', model=model, tokenizer=tokenizer)

ner_model(text)

Some weights of the model checkpoint at sagorsarker/codeswitch-spaeng-ner-lince were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'B-ORG',
  'score': 0.90762955,
  'index': 8,
  'word': 'NASA',
  'start': 16,
  'end': 20},
 {'entity': 'B-LOC',
  'score': 0.45779723,
  'index': 25,
  'word': 'Mars',
  'start': 96,
  'end': 100},
 {'entity': 'B-LOC',
  'score': 0.67780584,
  'index': 37,
  'word': 'Mars',
  'start': 152,
  'end': 156},
 {'entity': 'B-PROD',
  'score': 0.823176,
  'index': 60,
  'word': 'Viking',
  'start': 240,
  'end': 246},
 {'entity': 'I-PROD',
  'score': 0.6758249,
  'index': 61,
  'word': '1',
  'start': 247,
  'end': 248},
 {'entity': 'B-OTHER',
  'score': 0.5148923,
  'index': 98,
  'word': 'Mart',
  'start': 407,
  'end': 411},
 {'entity': 'B-OTHER',
  'score': 0.42019445,
  'index': 99,
  'word': '##ian',
  'start': 411,
  'end': 414},
 {'entity': 'B-LOC',
  'score': 0.76582533,
  'index': 104,
  'word': 'C',
  'start': 435,
  'end': 436},
 {'entity': 'B-LOC',
  'score': 0.74930793,
  'index': 105,
  'word': '##yd',
  'start': 436,
  'end': 438},
 {'entity': 'I-LOC',
  'score':

In [66]:
with open("6 sagorsarkercodeswitch-spaeng-ner-lince.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-GROUP    2
B-LOC      9
B-ORG      3
B-OTHER    4
B-PER      1
B-PROD     2
B-TIME     2
I-LOC      2
I-OTHER    1
I-PER      2
I-PROD     3
I-TIME     4
dtype: int64


entity   word    
B-GROUP  ##n         1
         Mali        1
B-LOC    ##yd        1
         American    1
         C           1
         Mars        6
B-ORG    NASA        3
B-OTHER  ##ian       1
         Egypt       1
         Mart        1
         Viking      1
B-PER    Michael     1
B-PROD   Mars        1
         Viking      1
B-TIME   April       2
I-LOC    ##onia      1
         West        1
I-OTHER  ##ion       1
I-PER    ##n         1
         Mali        1
I-PROD   ##biter     1
         1           1
         Or          1
I-TIME   ,           1
         1998        1
         5           1
         8           1
dtype: int64

## 7  gunghio/xlm-roberta-base-finetuned-panx-ner

In [68]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("gunghio/xlm-roberta-base-finetuned-panx-ner")
model = AutoModelForTokenClassification.from_pretrained("gunghio/xlm-roberta-base-finetuned-panx-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")
example = text

ner_results = nlp(example)
print(ner_results)

[{'entity_group': 'ORG', 'score': 0.9839335, 'word': 'NASA', 'start': 16, 'end': 20}, {'entity_group': 'ORG', 'score': 0.9257912, 'word': 'FaceOnMars,', 'start': 88, 'end': 101}, {'entity_group': 'LOC', 'score': 0.8524113, 'word': 'Mars,', 'start': 152, 'end': 157}, {'entity_group': 'ORG', 'score': 0.9470773, 'word': 'Viking1spacecraft', 'start': 240, 'end': 259}, {'entity_group': 'ORG', 'score': 0.6181985, 'word': 'Martian', 'start': 407, 'end': 414}, {'entity_group': 'LOC', 'score': 0.49310815, 'word': 'mesa,', 'start': 415, 'end': 420}, {'entity_group': 'LOC', 'score': 0.8700732, 'word': 'Cydonia,', 'start': 435, 'end': 443}, {'entity_group': 'ORG', 'score': 0.57100993, 'word': 'EgyptionPharaoh.', 'start': 496, 'end': 513}, {'entity_group': 'LOC', 'score': 0.46428245, 'word': 'formation', 'start': 622, 'end': 631}, {'entity_group': 'ORG', 'score': 0.90882486, 'word': "NASA's", 'start': 801, 'end': 807}, {'entity_group': 'LOC', 'score': 0.5685518, 'word': 'Mars--', 'start': 843, 'end

C:\Users\NW\anaconda3\Lib\site-packages\transformers\pipelines\token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [69]:

with open("7 gunghioxlm-roberta-base-finetuned-panx-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity_group']).size())
aux.groupby(['entity_group', 'word']).size()

entity_group
LOC     9
ORG    11
PER     1
dtype: int64


entity_group  word                 
LOC           AmericanWest.            1
              Cydonia,                 1
              Mars                     1
              Mars,                    2
              Mars--                   1
              Mars.                    1
              formation                1
              mesa,                    1
ORG           EgyptionPharaoh.         1
              FaceOnMars,              1
              MarsOrbitercamerateam    1
              Martian                  1
              NASA                     1
              NASA's                   1
              NASAbudget               1
              Viking1spacecraft        1
              Vikingphotos,            1
              ancientcivilization      1
              digitalimage,            1
PER           MichaelMalin             1
dtype: int64

## 8 51la5/roberta-large-NER

In [71]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("51la5/roberta-large-NER")
model = AutoModelForTokenClassification.from_pretrained("51la5/roberta-large-NER")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

C:\Users\NW\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at 51la5/roberta-large-NER were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_

[{'entity': 'I-ORG',
  'score': 0.9999913,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': 'I-MISC',
  'score': 0.9999641,
  'index': 23,
  'word': '▁Face',
  'start': 88,
  'end': 92},
 {'entity': 'I-MISC',
  'score': 0.99989665,
  'index': 24,
  'word': '▁On',
  'start': 93,
  'end': 95},
 {'entity': 'I-MISC',
  'score': 0.97350365,
  'index': 25,
  'word': '▁Mars',
  'start': 96,
  'end': 100},
 {'entity': 'I-LOC',
  'score': 0.9999362,
  'index': 36,
  'word': '▁Mars',
  'start': 152,
  'end': 156},
 {'entity': 'I-MISC',
  'score': 0.9992086,
  'index': 58,
  'word': '▁Viking',
  'start': 240,
  'end': 246},
 {'entity': 'I-MISC',
  'score': 0.9989502,
  'index': 59,
  'word': '▁1',
  'start': 247,
  'end': 248},
 {'entity': 'I-MISC',
  'score': 0.999977,
  'index': 97,
  'word': '▁Marti',
  'start': 407,
  'end': 412},
 {'entity': 'I-MISC',
  'score': 0.99619055,
  'index': 98,
  'word': 'an',
  'start': 412,
  'end': 414},
 {'entity': 'I-LOC',
  'score': 

In [72]:
with open("8 51la5roberta-large-NER.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
I-LOC      9
I-MISC    15
I-ORG      3
I-PER      3
dtype: int64


entity  word     
I-LOC   do           1
        nia          1
        ▁Cy          1
        ▁Mars        5
        ▁West        1
I-MISC  an           1
        bit          1
        er           1
        ion          1
        ▁1           1
        ▁American    1
        ▁Egypt       1
        ▁Face        1
        ▁Mars        2
        ▁Marti       1
        ▁On          1
        ▁Or          1
        ▁Viking      2
I-ORG   ▁NASA        3
I-PER   ▁Malin       2
        ▁Michael     1
dtype: int64

## 9 dmargutierrezdistilbert-base-multilingual-cased-mapa_coarse-ner

In [74]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("dmargutierrez/distilbert-base-multilingual-cased-mapa_coarse-ner")
model = AutoModelForTokenClassification.from_pretrained("dmargutierrez/distilbert-base-multilingual-cased-mapa_coarse-ner")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

[{'entity': 'B-ADDRESS',
  'score': 0.89953065,
  'index': 104,
  'word': 'C',
  'start': 435,
  'end': 436},
 {'entity': 'I-ADDRESS',
  'score': 0.776557,
  'index': 105,
  'word': '##yd',
  'start': 436,
  'end': 438},
 {'entity': 'I-ADDRESS',
  'score': 0.87639356,
  'index': 106,
  'word': '##onia',
  'start': 438,
  'end': 442},
 {'entity': 'B-AMOUNT',
  'score': 0.94337094,
  'index': 248,
  'word': '25',
  'start': 1013,
  'end': 1015},
 {'entity': 'I-AMOUNT',
  'score': 0.83502764,
  'index': 249,
  'word': 'years',
  'start': 1016,
  'end': 1021},
 {'entity': 'B-DATE',
  'score': 0.993107,
  'index': 311,
  'word': 'April',
  'start': 1297,
  'end': 1302},
 {'entity': 'I-DATE',
  'score': 0.9911287,
  'index': 312,
  'word': '5',
  'start': 1303,
  'end': 1304},
 {'entity': 'I-DATE',
  'score': 0.9842742,
  'index': 313,
  'word': ',',
  'start': 1304,
  'end': 1305},
 {'entity': 'I-DATE',
  'score': 0.9907127,
  'index': 314,
  'word': '1998',
  'start': 1306,
  'end': 1310},

In [75]:
with open("9 dmargutierrezdistilbert-base-multilingual-cased-mapa_coarse-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-ADDRESS         2
B-AMOUNT          1
B-DATE            2
B-ORGANISATION    1
B-PERSON          2
I-ADDRESS         3
I-AMOUNT          1
I-DATE            6
I-ORGANISATION    2
I-PERSON          3
dtype: int64


entity          word    
B-ADDRESS       American    1
                C           1
B-AMOUNT        25          1
B-DATE          April       2
B-ORGANISATION  Mars        1
B-PERSON        Mali        1
                Michael     1
I-ADDRESS       ##onia      1
                ##yd        1
                West        1
I-AMOUNT        years       1
I-DATE          ,           2
                1998        1
                2001        1
                5           1
                8           1
I-ORGANISATION  ##biter     1
                Or          1
I-PERSON        ##n         2
                Mali        1
dtype: int64

## 10 mbrutonspa_enpt_mBERT

In [77]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("mbruton/spa_enpt_mBERT")
model = AutoModelForTokenClassification.from_pretrained("mbruton/spa_enpt_mBERT")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

[{'entity': 'r0:arg1|tem',
  'score': 0.8560656,
  'index': 4,
  'word': 'you',
  'start': 7,
  'end': 10},
 {'entity': 'r0:root',
  'score': 0.99593973,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'r1:arg1|tem',
  'score': 0.7426193,
  'index': 11,
  'word': 'you',
  'start': 32,
  'end': 35},
 {'entity': 'r1:root',
  'score': 0.94838774,
  'index': 13,
  'word': 'be',
  'start': 43,
  'end': 45},
 {'entity': 'r1:arg2|atr',
  'score': 0.64391094,
  'index': 14,
  'word': 'able',
  'start': 46,
  'end': 50},
 {'entity': 'r2:root',
  'score': 0.80442923,
  'index': 16,
  'word': 'tell',
  'start': 54,
  'end': 58},
 {'entity': 'r1:arg1|pat',
  'score': 0.33438408,
  'index': 20,
  'word': 'story',
  'start': 72,
  'end': 77},
 {'entity': 'r4:arg1|tem',
  'score': 0.34810358,
  'index': 27,
  'word': 'which',
  'start': 102,
  'end': 107},
 {'entity': 'r5:root',
  'score': 0.5292147,
  'index': 30,
  'word': 'is',
  'start': 118,
  'end': 120},
 {'entity': 'r4:

In [78]:

with open("10 mbrutonspa_enpt_mBERT.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
r0:arg1|tem      1
r0:root          1
r10:root         5
r1:arg1|pat      1
r1:arg1|tem      1
r1:arg2|atr      1
r1:root          1
r2:root          1
r4:arg1|tem      1
r4:arg2|atr      1
r5:arg1|tem      1
r5:argM|fin      1
r5:root          2
r6:arg0|agt      4
r6:arg2|atr      1
r6:root          1
r7:arg0|agt      1
r7:arg1|pat      5
r7:arg1|tem      2
r7:arg2|atr      1
r7:root         10
r8:arg0|agt      8
r8:arg1|pat      4
r8:arg1|tem      8
r8:arg2|atr     18
r8:argM|tmp      3
r8:root          3
r9:arg1|pat     12
r9:root        102
dtype: int64


entity       word   
r0:arg1|tem  you        1
r0:root      re         1
r10:root     hidden     1
             in         1
             picture    1
                       ..
r9:root      what       1
             which      2
             would      1
             ye         1
             you        2
Length: 174, dtype: int64

## 11  benjamin/wtp-bert-mini

In [80]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-bert-mini")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-bert-mini")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

ValueError: The checkpoint you are trying to load has model type `bert-char` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 12 Babelscapewikineural-multilingual-ner

In [83]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

[{'entity': 'B-ORG',
  'score': 0.9951147,
  'index': 8,
  'word': 'NASA',
  'start': 16,
  'end': 20},
 {'entity': 'I-MISC',
  'score': 0.6191017,
  'index': 23,
  'word': 'Face',
  'start': 88,
  'end': 92},
 {'entity': 'I-MISC',
  'score': 0.5708344,
  'index': 24,
  'word': 'On',
  'start': 93,
  'end': 95},
 {'entity': 'I-MISC',
  'score': 0.5786274,
  'index': 25,
  'word': 'Mars',
  'start': 96,
  'end': 100},
 {'entity': 'B-LOC',
  'score': 0.877606,
  'index': 37,
  'word': 'Mars',
  'start': 152,
  'end': 156},
 {'entity': 'I-MISC',
  'score': 0.9051992,
  'index': 60,
  'word': 'Viking',
  'start': 240,
  'end': 246},
 {'entity': 'I-MISC',
  'score': 0.9834109,
  'index': 61,
  'word': '1',
  'start': 247,
  'end': 248},
 {'entity': 'I-MISC',
  'score': 0.48295248,
  'index': 98,
  'word': 'Mart',
  'start': 407,
  'end': 411},
 {'entity': 'I-MISC',
  'score': 0.47647634,
  'index': 99,
  'word': '##ian',
  'start': 411,
  'end': 414},
 {'entity': 'B-LOC',
  'score': 0.97810

In [84]:

with open("12 Babelscapewikineural-multilingual-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC      7
B-ORG      3
B-PER      2
I-LOC      2
I-MISC    15
I-PER      4
dtype: int64


entity  word    
B-LOC   C           1
        Egypt       1
        Mars        5
B-ORG   NASA        3
B-PER   Mali        1
        Michael     1
I-LOC   ##onia      1
        ##yd        1
I-MISC  ##biter     1
        ##ian       1
        ##ion       1
        1           1
        American    1
        Face        1
        Mars        2
        Mart        1
        On          1
        Or          1
        Ph          1
        Viking      2
        West        1
I-PER   ##ara       1
        ##n         2
        Mali        1
dtype: int64

## 13 julian-schelb/roberta-ner-multilingual

In [88]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("julian-schelb/roberta-ner-multilingual")
model = AutoModelForTokenClassification.from_pretrained("julian-schelb/roberta-ner-multilingual")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

[{'entity': 'B-ORG',
  'score': 0.8837392,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': 'B-ORG',
  'score': 0.38925776,
  'index': 97,
  'word': '▁Marti',
  'start': 407,
  'end': 412},
 {'entity': 'B-LOC',
  'score': 0.7155649,
  'index': 103,
  'word': '▁Cy',
  'start': 435,
  'end': 437},
 {'entity': 'B-LOC',
  'score': 0.64458394,
  'index': 104,
  'word': 'do',
  'start': 437,
  'end': 439},
 {'entity': 'B-LOC',
  'score': 0.623109,
  'index': 105,
  'word': 'nia',
  'start': 439,
  'end': 442},
 {'entity': 'B-ORG',
  'score': 0.34994408,
  'index': 119,
  'word': '▁Egypt',
  'start': 496,
  'end': 501},
 {'entity': 'B-ORG',
  'score': 0.32543704,
  'index': 120,
  'word': 'ion',
  'start': 501,
  'end': 504},
 {'entity': 'I-PER',
  'score': 0.4674562,
  'index': 121,
  'word': '▁Phar',
  'start': 505,
  'end': 509},
 {'entity': 'I-PER',
  'score': 0.53341544,
  'index': 122,
  'word': 'a',
  'start': 509,
  'end': 510},
 {'entity': 'I-PER',
  'score':

In [89]:
with open("13 julian-schelbroberta-ner-multilingual.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC    4
B-ORG    7
B-PER    1
I-LOC    1
I-ORG    3
I-PER    4
dtype: int64


entity  word     
B-LOC   do           1
        nia          1
        ▁American    1
        ▁Cy          1
B-ORG   ion          1
        ▁Egypt       1
        ▁Mars        1
        ▁Marti       1
        ▁NASA        3
B-PER   ▁Michael     1
I-LOC   ▁West        1
I-ORG   bit          1
        er           1
        ▁Or          1
I-PER   a            1
        oh           1
        ▁Malin       1
        ▁Phar        1
dtype: int64

## 14 FacebookAI/xlm-roberta-large-finetuned-conll03-german

In [91]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large-finetuned-conll03-german")
model = AutoModelForTokenClassification.from_pretrained("FacebookAI/xlm-roberta-large-finetuned-conll03-english")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)


Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-ORG', 'score': 0.9999913, 'index': 8, 'word': '▁NASA', 'start': 16, 'end': 20}, {'entity': 'I-MISC', 'score': 0.9999641, 'index': 23, 'word': '▁Face', 'start': 88, 'end': 92}, {'entity': 'I-MISC', 'score': 0.99989665, 'index': 24, 'word': '▁On', 'start': 93, 'end': 95}, {'entity': 'I-MISC', 'score': 0.97350365, 'index': 25, 'word': '▁Mars', 'start': 96, 'end': 100}, {'entity': 'I-LOC', 'score': 0.9999362, 'index': 36, 'word': '▁Mars', 'start': 152, 'end': 156}, {'entity': 'I-MISC', 'score': 0.9992086, 'index': 58, 'word': '▁Viking', 'start': 240, 'end': 246}, {'entity': 'I-MISC', 'score': 0.9989502, 'index': 59, 'word': '▁1', 'start': 247, 'end': 248}, {'entity': 'I-MISC', 'score': 0.999977, 'index': 97, 'word': '▁Marti', 'start': 407, 'end': 412}, {'entity': 'I-MISC', 'score': 0.99619055, 'index': 98, 'word': 'an', 'start': 412, 'end': 414}, {'entity': 'I-LOC', 'score': 0.9999354, 'index': 103, 'word': '▁Cy', 'start': 435, 'end': 437}, {'entity': 'I-LOC', 'score': 0.999

In [92]:
with open("14 FacebookAIxlm-roberta-large-finetuned-conll03-german.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
I-LOC      9
I-MISC    15
I-ORG      3
I-PER      3
dtype: int64


entity  word     
I-LOC   do           1
        nia          1
        ▁Cy          1
        ▁Mars        5
        ▁West        1
I-MISC  an           1
        bit          1
        er           1
        ion          1
        ▁1           1
        ▁American    1
        ▁Egypt       1
        ▁Face        1
        ▁Mars        2
        ▁Marti       1
        ▁On          1
        ▁Or          1
        ▁Viking      2
I-ORG   ▁NASA        3
I-PER   ▁Malin       2
        ▁Michael     1
dtype: int64

## 15  jplu/tf-xlm-r-ner-40-lang

In [103]:
pip install transformers[sentencepiece]

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import pipeline

nlp_ner = pipeline(
    "ner",
    model="jplu/tf-xlm-r-ner-40-lang",
    tokenizer=(
        'jplu/tf-xlm-r-ner-40-lang'),
    framework="tf"
)

nlp_ner(text)



Some layers from the model checkpoint at jplu/tf-xlm-r-ner-40-lang were not used when initializing TFXLMRobertaForTokenClassification: ['dropout_38']
- This IS expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaForTokenClassification were initialized from the model checkpoint at jplu/tf-xlm-r-ner-40-lang.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForTokenClassification for predictions without further training.
Asking to truncate to max_length but no maxi

[{'entity': 'ORG',
  'score': 0.94845986,
  'index': 8,
  'word': '▁NASA',
  'start': 15,
  'end': 20},
 {'entity': 'ORG',
  'score': 0.7998288,
  'index': 58,
  'word': '▁Viking',
  'start': 239,
  'end': 246},
 {'entity': 'ORG',
  'score': 0.7579509,
  'index': 59,
  'word': '▁1',
  'start': 246,
  'end': 248},
 {'entity': 'ORG',
  'score': 0.52262145,
  'index': 60,
  'word': '▁space',
  'start': 248,
  'end': 254},
 {'entity': 'ORG',
  'score': 0.5880066,
  'index': 193,
  'word': '▁NASA',
  'start': 800,
  'end': 805},
 {'entity': 'ORG',
  'score': 0.7987309,
  'index': 285,
  'word': '▁NASA',
  'start': 1168,
  'end': 1173},
 {'entity': 'PER',
  'score': 0.9565463,
  'index': 319,
  'word': '▁Michael',
  'start': 1311,
  'end': 1319},
 {'entity': 'PER',
  'score': 0.9528012,
  'index': 320,
  'word': '▁Malin',
  'start': 1319,
  'end': 1325},
 {'entity': 'ORG',
  'score': 0.5696624,
  'index': 323,
  'word': '▁Mars',
  'start': 1333,
  'end': 1338},
 {'entity': 'LOC',
  'score': 

In [7]:
with open("15  jplutf-xlm-r-ner-40-lang.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
LOC    2
ORG    7
PER    2
dtype: int64


entity  word     
LOC     ▁American    1
        ▁West        1
ORG     ▁1           1
        ▁Mars        1
        ▁NASA        3
        ▁Viking      1
        ▁space       1
PER     ▁Malin       1
        ▁Michael     1
dtype: int64

## 16 sagorsarker/codeswitch-spaeng-lid-lince

In [127]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("sagorsarker/codeswitch-spaeng-lid-lince")

model = AutoModelForTokenClassification.from_pretrained("sagorsarker/codeswitch-spaeng-lid-lince")
lid_model = pipeline('ner', model=model, tokenizer=tokenizer)

lid_model(text)

Some weights of the model checkpoint at sagorsarker/codeswitch-spaeng-lid-lince were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'en',
  'score': 0.9998629,
  'index': 1,
  'word': 'So',
  'start': 0,
  'end': 2},
 {'entity': 'other',
  'score': 0.9999267,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'en',
  'score': 0.99985707,
  'index': 3,
  'word': 'if',
  'start': 4,
  'end': 6},
 {'entity': 'en',
  'score': 0.99984396,
  'index': 4,
  'word': 'you',
  'start': 7,
  'end': 10},
 {'entity': 'en',
  'score': 0.9998392,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'en',
  'score': 0.9998178,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'en',
  'score': 0.99961334,
  'index': 7,
  'word': 'a',
  'start': 14,
  'end': 15},
 {'entity': 'ne',
  'score': 0.99565876,
  'index': 8,
  'word': 'NASA',
  'start': 16,
  'end': 20},
 {'entity': 'en',
  'score': 0.9997341,
  'index': 9,
  'word': 'scientist',
  'start': 21,
  'end': 30},
 {'entity': 'other',
  'score': 0.99992585,
  'index': 10,
  'word': ',',
  'start': 30,
  'end': 31},

In [130]:
with open("16 sagorsarkercodeswitch-spaeng-lid-lince.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()


aux.groupby(['entity']) \
       .agg({'entity':'size', 'score':'mean'}) 


entity
en       411
ne        22
other     61
dtype: int64


,entity,score
entity,,
en,411,0.996325
ne,22,0.925739
other,61,0.990110


## 17 jvdzwaan/ocrpostcorrection-task-1

In [10]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("jvdzwaan/ocrpostcorrection-task-1")
model = AutoModelForTokenClassification.from_pretrained("jvdzwaan/ocrpostcorrection-task-1")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

C:\Users\NW\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

C:\Users\NW\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NW\.cache\huggingface\hub\models--jvdzwaan--ocrpostcorrection-task-1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OSError: Can't load tokenizer for 'jvdzwaan/ocrpostcorrection-task-1'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'jvdzwaan/ocrpostcorrection-task-1' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

## 18 GEOcite/AuthorParserModel

In [13]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("GEOcite/AuthorParserModel")
model = AutoModelForTokenClassification.from_pretrained("GEOcite/AuthorParserModel")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

[{'entity': 'B-ORG', 'score': 0.56347305, 'index': 8, 'word': 'nasa', 'start': 16, 'end': 20}, {'entity': 'I-ORG', 'score': 0.64977086, 'index': 25, 'word': 'mars', 'start': 96, 'end': 100}, {'entity': 'I-ORG', 'score': 0.5621034, 'index': 37, 'word': 'mars', 'start': 152, 'end': 156}, {'entity': 'B-ORG', 'score': 0.83589464, 'index': 59, 'word': 'viking', 'start': 240, 'end': 246}, {'entity': 'I-ORG', 'score': 0.93603927, 'index': 60, 'word': '1', 'start': 247, 'end': 248}, {'entity': 'I-ORG', 'score': 0.67972714, 'index': 61, 'word': 'spacecraft', 'start': 249, 'end': 259}, {'entity': 'B-LOC', 'score': 0.47661397, 'index': 97, 'word': 'marti', 'start': 407, 'end': 412}, {'entity': 'I-ORG', 'score': 0.5451927, 'index': 98, 'word': '##an', 'start': 412, 'end': 414}, {'entity': 'I-ORG', 'score': 0.60608196, 'index': 99, 'word': 'mesa', 'start': 415, 'end': 419}, {'entity': 'B-LOC', 'score': 0.8834184, 'index': 103, 'word': 'c', 'start': 435, 'end': 436}, {'entity': 'I-LOC', 'score': 0.7

In [14]:

with open("18 GEOciteAuthorParserModel.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC     5
B-ORG     9
B-PER     1
I-LOC     3
I-ORG    29
I-PER     2
dtype: int64


entity  word        
B-LOC   american        1
        butte           1
        c               1
        mars            1
        marti           1
B-ORG   absolute        1
        defenders       1
        egypt           1
        haunted         1
        mars            1
        nasa            2
        viking          2
B-PER   michael         1
I-LOC   ##onia          1
        ##yd            1
        west            1
I-ORG   ##an            1
        ##h             1
        ##ion           1
        ##oce           1
        ##r             1
        ##rao           1
        ##ry            1
        1               1
        camera          1
        civilization    1
        gr              1
        mars            5
        maximum         1
        mesa            2
        nasa            1
        of              1
        on              1
        or              1
        orbite          1
        pha             1
        revolution      1
        spacecraf

## 19 mbruton/spa_en_XLM-R

In [16]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("mbruton/spa_en_XLM-R")
model = AutoModelForTokenClassification.from_pretrained("mbruton/spa_en_XLM-R")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

[{'entity': 'r0:arg1|tem', 'score': 0.9885584, 'index': 4, 'word': '▁you', 'start': 6, 'end': 10}, {'entity': 'r0:root', 'score': 0.9828293, 'index': 5, 'word': "'", 'start': 10, 'end': 11}, {'entity': 'r0:root', 'score': 0.9987388, 'index': 6, 'word': 're', 'start': 11, 'end': 13}, {'entity': 'r0:arg2|atr', 'score': 0.9798707, 'index': 9, 'word': '▁scientist', 'start': 20, 'end': 30}, {'entity': 'r1:arg1|tem', 'score': 0.9950965, 'index': 11, 'word': '▁you', 'start': 31, 'end': 35}, {'entity': 'r1:root', 'score': 0.9981749, 'index': 13, 'word': '▁be', 'start': 42, 'end': 45}, {'entity': 'r1:arg2|atr', 'score': 0.98995215, 'index': 14, 'word': '▁able', 'start': 45, 'end': 50}, {'entity': 'r2:root', 'score': 0.99103266, 'index': 16, 'word': '▁tell', 'start': 53, 'end': 58}, {'entity': 'r2:arg2|ben', 'score': 0.9612953, 'index': 17, 'word': '▁me', 'start': 58, 'end': 61}, {'entity': 'r2:arg1|pat', 'score': 0.98190963, 'index': 20, 'word': '▁story', 'start': 71, 'end': 77}, {'entity': 'r3

In [23]:

with open("19 mbrutonspa_en_XLM-R.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
#print(aux.groupby(['entity']).size())
aux2 = aux.groupby(['entity', 'word']).size()
aux2


entity       word      
r0:arg1|tem  ▁you          1
r0:arg2|atr  ▁scientist    1
r0:root                    1
             re            1
r10:root     d             1
                          ..
r9:root      ▁there        1
             ▁thought      1
             ▁was          2
             ▁wasn         1
             ▁were         2
Length: 168, dtype: int64

In [27]:
pd.set_option('display.max_rows', None)  

In [28]:
aux2

entity       word      
r0:arg1|tem  ▁you          1
r0:arg2|atr  ▁scientist    1
r0:root                    1
             re            1
r10:root     d             1
             med           1
             ▁capture      1
             ▁made         1
             ▁was          1
r11:root     d             1
             ed            1
             mble          1
             ▁for          1
             ▁rese         1
             ▁reveal       1
             ▁using        1
             ▁was          1
r1:arg1|tem  ▁you          1
r1:arg2|atr  ▁able         1
r1:root      ▁be           1
r2:arg1|pat  ▁story        1
r2:arg2|ben  ▁me           1
r2:root      ▁shot         1
             ▁tell         1
r3:arg1|tem  ▁which        1
r3:arg2|atr  ▁evidence     1
r3:argM|adv  ▁obviously    1
r3:argM|tmp  ▁for          1
r3:root      ▁appeared     1
             ▁is           1
r4:arg0|agt  ▁people       1
r4:arg1|tem  ▁life         1
r4:arg2|loc  ▁in           1
r4:argM|loc  ▁in   

## 20 mbruton/gal_enptsp_mBERT

In [30]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("mbruton/gal_enptsp_mBERT")
model = AutoModelForTokenClassification.from_pretrained("mbruton/gal_enptsp_mBERT")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

[{'entity': 'r0:root', 'score': 0.69288, 'index': 6, 'word': 're', 'start': 11, 'end': 13}, {'entity': 'r1:root', 'score': 0.8305716, 'index': 16, 'word': 'tell', 'start': 54, 'end': 58}, {'entity': 'r2:arg1', 'score': 0.65180844, 'index': 20, 'word': 'story', 'start': 72, 'end': 77}, {'entity': 'r5:root', 'score': 0.4588822, 'index': 44, 'word': 'created', 'start': 180, 'end': 187}, {'entity': 'r6:root', 'score': 0.49542937, 'index': 64, 'word': 'ci', 'start': 264, 'end': 266}, {'entity': 'r6:root', 'score': 0.45653778, 'index': 65, 'word': '##rc', 'start': 266, 'end': 268}, {'entity': 'r6:root', 'score': 0.31713662, 'index': 66, 'word': '##ling', 'start': 268, 'end': 272}, {'entity': 'r6:arg1', 'score': 0.37775016, 'index': 68, 'word': 'planet', 'start': 277, 'end': 283}, {'entity': 'r6:root', 'score': 0.41948485, 'index': 70, 'word': 'sna', 'start': 285, 'end': 288}, {'entity': 'r6:root', 'score': 0.28788853, 'index': 71, 'word': '##pping', 'start': 288, 'end': 293}, {'entity': 'r6:

In [31]:
with open("20 mbrutongal_enptsp_mBERT.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
r0:root    1
r1:root    1
r2:arg1    1
r2:root    2
r4:root    1
r5:root    3
r6:arg1    3
r6:root    5
r7:arg1    1
r8:root    7
dtype: int64


entity   word     
r0:root  re           1
r1:root  tell         1
r2:arg1  story        1
r2:root  make         1
         wish         1
r4:root  take         1
r5:root  created      1
         hide         1
         thought      1
r6:arg1  it           1
         photos       1
         planet       1
r6:root  ##ling       1
         ##pping      1
         ##rc         1
         ci           1
         sna          1
r7:arg1  picture      1
r8:root  announced    1
         appeared     1
         figure       1
         formed       1
         reveal       1
         spotted      1
         took         1
dtype: int64

## 21 benjamin/wtp-bert-tiny

In [33]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-bert-tiny")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-bert-tiny")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

ValueError: The checkpoint you are trying to load has model type `bert-char` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 22 benjamin/wtp-canine-s-1l

In [35]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-1l")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-1l")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 23 benjamin/wtp-canine-s-6l

In [37]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-6l")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-6l")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 24 benjamin/wtp-canine-s-9l

In [39]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-9l")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-9l")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 25 benjamin/wtp-canine-s-1l-no-adapters

In [41]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-1l-no-adapters")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-1l-no-adapters")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 26 benjamin/wtp-canine-s-6l-no-adapters

In [44]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-6l-no-adapters")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-6l-no-adapters")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 27 benjamin/wtp-canine-s-12l-no-adapters

In [46]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-12l-no-adapters")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-12l-no-adapters")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 28 Posos/ClinicalNER

In [48]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("Posos/ClinicalNER")
model = AutoModelForTokenClassification.from_pretrained("Posos/ClinicalNER")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

[]


## 29 numind/NuNER-multilingual-v0.1

In [50]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("numind/NuNER-multilingual-v0.1")
model = AutoModelForTokenClassification.from_pretrained("numind/NuNER-multilingual-v0.1")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at numind/NuNER-multilingual-v0.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'entity': 'LABEL_1', 'score': 0.6517681, 'index': 1, 'word': 'So', 'start': 0, 'end': 2}, {'entity': 'LABEL_1', 'score': 0.6665356, 'index': 2, 'word': ',', 'start': 2, 'end': 3}, {'entity': 'LABEL_1', 'score': 0.6989242, 'index': 3, 'word': 'if', 'start': 4, 'end': 6}, {'entity': 'LABEL_1', 'score': 0.7015521, 'index': 4, 'word': 'you', 'start': 7, 'end': 10}, {'entity': 'LABEL_1', 'score': 0.69064546, 'index': 5, 'word': "'", 'start': 10, 'end': 11}, {'entity': 'LABEL_1', 'score': 0.6504881, 'index': 6, 'word': 're', 'start': 11, 'end': 13}, {'entity': 'LABEL_1', 'score': 0.57260615, 'index': 7, 'word': 'a', 'start': 14, 'end': 15}, {'entity': 'LABEL_1', 'score': 0.5917183, 'index': 8, 'word': 'NASA', 'start': 16, 'end': 20}, {'entity': 'LABEL_1', 'score': 0.5965599, 'index': 9, 'word': 'scientist', 'start': 21, 'end': 30}, {'entity': 'LABEL_1', 'score': 0.6810555, 'index': 10, 'word': ',', 'start': 30, 'end': 31}, {'entity': 'LABEL_1', 'score': 0.71639186, 'index': 11, 'word': 'yo

In [51]:
with open("29 numindNuNER-multilingual-v0.1.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
LABEL_0    188
LABEL_1    306
dtype: int64


entity   word      
LABEL_0                 1
         ##ara          1
         ##con          1
         ##er           1
         ##form         3
         ##ian          1
         ##ing          1
         ##ion          1
         ##less         1
         ##ling         1
         ##oce          1
         ##oh           1
         ##onia         1
         ##or           1
         ##out          1
         ##rc           1
         ##ry           1
         ##s            3
         ##te           1
         ##yd           1
         ##zation       1
         ##ze           1
         ,              8
         .              9
         1              1
         25             1
         5              1
         8              1
         American       1
         April          2
         C              1
         Egypt          1
         Mars           4
         Mart           1
         Ph             1
         Very           1
         Viking         1
         West     

## 30 orgcatorg/bert-base-multilingual-cased-ner

In [53]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("orgcatorg/bert-base-multilingual-cased-ner")
model = AutoModelForTokenClassification.from_pretrained("orgcatorg/bert-base-multilingual-cased-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'B-ORG',
  'score': 0.99468493,
  'index': 60,
  'word': 'Viking',
  'start': 240,
  'end': 246},
 {'entity': 'I-ORG',
  'score': 0.99335337,
  'index': 61,
  'word': '1',
  'start': 247,
  'end': 248},
 {'entity': 'B-LOC',
  'score': 0.9839153,
  'index': 104,
  'word': 'C',
  'start': 435,
  'end': 436},
 {'entity': 'B-LOC',
  'score': 0.9820802,
  'index': 105,
  'word': '##yd',
  'start': 436,
  'end': 438},
 {'entity': 'B-LOC',
  'score': 0.9102487,
  'index': 106,
  'word': '##onia',
  'start': 438,
  'end': 442},
 {'entity': 'B-PER',
  'score': 0.8671783,
  'index': 121,
  'word': 'Egypt',
  'start': 496,
  'end': 501},
 {'entity': 'B-PER',
  'score': 0.7123952,
  'index': 122,
  'word': '##ion',
  'start': 501,
  'end': 504},
 {'entity': 'I-PER',
  'score': 0.94853634,
  'index': 123,
  'word': 'Ph',
  'start': 505,
  'end': 507},
 {'entity': 'I-PER',
  'score': 0.96879715,
  'index': 124,
  'word': '##ara',
  'start': 507,
  'end': 510},
 {'entity': 'I-PER',
  'sco

In [55]:
with open("30 orgcatorgbert-base-multilingual-cased-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC    4
B-ORG    2
B-PER    3
I-LOC    1
I-ORG    3
I-PER    5
dtype: int64


entity  word    
B-LOC   ##onia      1
        ##yd        1
        American    1
        C           1
B-ORG   Mars        1
        Viking      1
B-PER   ##ion       1
        Egypt       1
        Michael     1
I-LOC   West        1
I-ORG   ##biter     1
        1           1
        Or          1
I-PER   ##ara       1
        ##n         1
        ##oh        1
        Mali        1
        Ph          1
dtype: int64

## 31 orgcatorg/xlm-roberta-base-ner

In [57]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("orgcatorg/xlm-roberta-base-ner")
model = AutoModelForTokenClassification.from_pretrained("orgcatorg/xlm-roberta-base-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'B-ORG',
  'score': 0.9307573,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': 'I-ORG',
  'score': 0.55268466,
  'index': 9,
  'word': '▁scientist',
  'start': 21,
  'end': 30},
 {'entity': 'B-ORG',
  'score': 0.8743138,
  'index': 58,
  'word': '▁Viking',
  'start': 240,
  'end': 246},
 {'entity': 'I-ORG',
  'score': 0.8924012,
  'index': 59,
  'word': '▁1',
  'start': 247,
  'end': 248},
 {'entity': 'I-ORG',
  'score': 0.8749563,
  'index': 60,
  'word': '▁space',
  'start': 249,
  'end': 254},
 {'entity': 'I-ORG',
  'score': 0.83837587,
  'index': 61,
  'word': 'craft',
  'start': 254,
  'end': 259},
 {'entity': 'B-ORG',
  'score': 0.31897616,
  'index': 97,
  'word': '▁Marti',
  'start': 407,
  'end': 412},
 {'entity': 'B-LOC',
  'score': 0.66254807,
  'index': 103,
  'word': '▁Cy',
  'start': 435,
  'end': 437},
 {'entity': 'B-LOC',
  'score': 0.6462039,
  'index': 104,
  'word': 'do',
  'start': 437,
  'end': 439},
 {'entity': 'B-LOC',
  'sco

In [58]:
with open("31 orgcatorgxlm-roberta-base-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC     5
B-ORG     6
B-PER     1
I-LOC     1
I-ORG    10
I-PER     1
dtype: int64


entity  word      
B-LOC   do            1
        nia           1
        ▁American     1
        ▁Cy           1
        ▁but          1
B-ORG   ion           1
        ▁Egypt        1
        ▁Mars         1
        ▁Marti        1
        ▁NASA         1
        ▁Viking       1
B-PER   ▁Michael      1
I-LOC   ▁West         1
I-ORG   a             1
        bit           1
        craft         1
        er            1
        oh            1
        ▁1            1
        ▁Or           1
        ▁Phar         1
        ▁scientist    1
        ▁space        1
I-PER   ▁Malin        1
dtype: int64

## 32 orgcatorg/EntityCS-39-PEP_MS_MLM-xlmr-base

In [60]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("orgcatorg/EntityCS-39-PEP_MS_MLM-xlmr-base")
model = AutoModelForTokenClassification.from_pretrained("orgcatorg/EntityCS-39-PEP_MS_MLM-xlmr-base")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-ORG',
  'score': 0.9218857,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': 'I-ORG',
  'score': 0.9510029,
  'index': 9,
  'word': '▁scientist',
  'start': 21,
  'end': 30},
 {'entity': 'I-ORG',
  'score': 0.8217206,
  'index': 25,
  'word': '▁Mars',
  'start': 96,
  'end': 100},
 {'entity': 'B-ORG',
  'score': 0.96086377,
  'index': 58,
  'word': '▁Viking',
  'start': 240,
  'end': 246},
 {'entity': 'I-ORG',
  'score': 0.9884774,
  'index': 59,
  'word': '▁1',
  'start': 247,
  'end': 248},
 {'entity': 'I-ORG',
  'score': 0.9854343,
  'index': 60,
  'word': '▁space',
  'start': 249,
  'end': 254},
 {'entity': 'I-ORG',
  'score': 0.97897524,
  'index': 61,
  'word': 'craft',
  'start': 254,
  'end': 259},
 {'entity': 'B-ORG',
  'score': 0.8928362,
  'index': 97,
  'word': '▁Marti',
  'start': 407,
  'end': 412},
 {'entity': 'B-ORG',
  'score': 0.83326113,
  'index': 98,
  'word': 'an',
  'start': 412,
  'end': 414},
 {'entity': 'I-ORG',
  'score

In [61]:

with open("32 orgcatorgEntityCS-39-PEP_MS_MLM-xlmr-base.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-ORG     6
B-PER     1
I-ORG    12
I-PER     1
dtype: int64


entity  word      
B-ORG   an            1
        ion           1
        ▁Egypt        1
        ▁Marti        1
        ▁NASA         1
        ▁Viking       1
B-PER   ▁Michael      1
I-ORG   a             1
        bit           1
        craft         1
        er            1
        oh            1
        ▁1            1
        ▁Mars         1
        ▁Or           1
        ▁Phar         1
        ▁mesa         1
        ▁scientist    1
        ▁space        1
I-PER   ▁Malin        1
dtype: int64

## 33 igorsterner/xlmr-multilingual-sentence-segmentation

In [63]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("igorsterner/xlmr-multilingual-sentence-segmentation")
model = AutoModelForTokenClassification.from_pretrained("igorsterner/xlmr-multilingual-sentence-segmentation")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': '|',
  'score': 0.9995815,
  'index': 49,
  'word': '?"',
  'start': 206,
  'end': 208},
 {'entity': '|',
  'score': 0.9974347,
  'index': 85,
  'word': '.',
  'start': 354,
  'end': 355},
 {'entity': '|',
  'score': 0.9996898,
  'index': 124,
  'word': '.',
  'start': 512,
  'end': 513},
 {'entity': '|',
  'score': 0.9997476,
  'index': 174,
  'word': '.',
  'start': 710,
  'end': 711},
 {'entity': '|',
  'score': 0.9997371,
  'index': 211,
  'word': '.',
  'start': 860,
  'end': 861},
 {'entity': '|',
  'score': 0.9997018,
  'index': 251,
  'word': '.',
  'start': 1021,
  'end': 1022},
 {'entity': '|',
  'score': 0.9995659,
  'index': 296,
  'word': '.',
  'start': 1224,
  'end': 1225},
 {'entity': '|',
  'score': 0.9988242,
  'index': 356,
  'word': '.',
  'start': 1493,
  'end': 1494},
 {'entity': '|',
  'score': 0.9983346,
  'index': 380,
  'word': '"',
  'start': 1590,
  'end': 1591},
 {'entity': '|',
  'score': 0.999102,
  'index': 415,
  'word': '.',
  'start': 1743

In [65]:

with open("33 igorsternerxlmr-multilingual-sentence-segmentation.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
|    13
dtype: int64


entity  word
|                1
        .       11
        ?        1
dtype: int64

## 34 mukowaty/punctuate-16

In [67]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("mukowaty/punctuate-16")
model = AutoModelForTokenClassification.from_pretrained("mukowaty/punctuate-16")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

C:\Users\NW\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': '0',
  'score': 0.77220947,
  'index': 1,
  'word': '▁So',
  'start': 0,
  'end': 2},
 {'entity': '0',
  'score': 0.9685962,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': '0',
  'score': 0.99998283,
  'index': 3,
  'word': '▁if',
  'start': 4,
  'end': 6},
 {'entity': '0',
  'score': 0.99998665,
  'index': 4,
  'word': '▁you',
  'start': 7,
  'end': 10},
 {'entity': '0',
  'score': 0.99998367,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': '0',
  'score': 0.99998736,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': '0',
  'score': 0.99998903,
  'index': 7,
  'word': '▁a',
  'start': 14,
  'end': 15},
 {'entity': '0',
  'score': 0.9999871,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': '0',
  'score': 0.8388357,
  'index': 9,
  'word': '▁scientist',
  'start': 21,
  'end': 30},
 {'entity': '0',
  'score': 0.9971527,
  'index': 10,
  'word': ',',
  'start': 30,
  'end': 31},
 {'entit

In [68]:


with open("34 mukowatypunctuate-16.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
,      5
.      5
0    483
dtype: int64


entity  word       
,       ▁No             1
        ▁no             1
        ▁size           1
        ▁wrong          1
        ▁yes            1
.                       1
        ?               1
        ze              1
        ▁1998.          1
        ▁ha             1
0                       6
        ,              34
        -               2
        .              11
        ;               1
        But             1
        a               1
        an              1
        bit             1
        cer             1
        cient           1
        craft           1
        ct              1
        d               3
        de              1
        do              1
        ed              1
        er              2
        form            3
        ing             1
        ings            1
        ion             1
        less            1
        ling            1
        mble            1
        med             1
        n               1
        ness      

## 35 HiTZ/mbert-argmining-abstrct-multilingual

In [70]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("HiTZ/mbert-argmining-abstrct-multilingual")
model = AutoModelForTokenClassification.from_pretrained("HiTZ/mbert-argmining-abstrct-multilingual")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[]

## 36 benjamin/wtp-canine-s-12l

In [72]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-12l")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-12l")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 37 benjamin/wtp-canine-s-3l-no-adapters

In [74]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-3l-no-adapters")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-3l-no-adapters")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 38 benjamin/wtp-canine-s-9l-no-adapters

In [77]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("benjamin/wtp-canine-s-9l-no-adapters")
model = AutoModelForTokenClassification.from_pretrained("benjamin/wtp-canine-s-9l-no-adapters")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

ValueError: The checkpoint you are trying to load has model type `la-canine` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 39 msislam/code-mixed-language-detection-XLMRoberta

In [79]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("msislam/code-mixed-language-detection-XLMRoberta")
model = AutoModelForTokenClassification.from_pretrained("msislam/code-mixed-language-detection-XLMRoberta")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'I-EN',
  'score': 0.9999919,
  'index': 1,
  'word': '▁So',
  'start': 0,
  'end': 2},
 {'entity': 'I-EN',
  'score': 0.99999356,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'I-EN',
  'score': 0.999995,
  'index': 3,
  'word': '▁if',
  'start': 4,
  'end': 6},
 {'entity': 'I-EN',
  'score': 0.99999523,
  'index': 4,
  'word': '▁you',
  'start': 7,
  'end': 10},
 {'entity': 'I-EN',
  'score': 0.9999944,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'I-EN',
  'score': 0.99999535,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'I-EN',
  'score': 0.9999951,
  'index': 7,
  'word': '▁a',
  'start': 14,
  'end': 15},
 {'entity': 'I-EN',
  'score': 0.9999945,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': 'I-EN',
  'score': 0.999995,
  'index': 9,
  'word': '▁scientist',
  'start': 21,
  'end': 30},
 {'entity': 'I-EN',
  'score': 0.99999547,
  'index': 10,
  'word': ',',
  'start': 

In [131]:

with open("39 msislamcode-mixed-language-detection-XLMRoberta.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()


aux.groupby(['entity']) \
       .agg({'entity':'size', 'score':'mean'})

entity
I-EN    493
dtype: int64


,entity,score
entity,,
I-EN,493,0.999995


## 40 DunnBC22/bert-base-multilingual-cased-fine_tuned-ner-WikiNeural_Multilingual

In [82]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("DunnBC22/bert-base-multilingual-cased-fine_tuned-ner-WikiNeural_Multilingual")
model = AutoModelForTokenClassification.from_pretrained("DunnBC22/bert-base-multilingual-cased-fine_tuned-ner-WikiNeural_Multilingual")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results



[{'entity': 'B-ORG',
  'score': 0.98604935,
  'index': 8,
  'word': 'NASA',
  'start': 16,
  'end': 20},
 {'entity': 'I-MISC',
  'score': 0.67684585,
  'index': 23,
  'word': 'Face',
  'start': 88,
  'end': 92},
 {'entity': 'I-MISC',
  'score': 0.6898482,
  'index': 24,
  'word': 'On',
  'start': 93,
  'end': 95},
 {'entity': 'I-MISC',
  'score': 0.72657937,
  'index': 25,
  'word': 'Mars',
  'start': 96,
  'end': 100},
 {'entity': 'B-LOC',
  'score': 0.8695399,
  'index': 37,
  'word': 'Mars',
  'start': 152,
  'end': 156},
 {'entity': 'I-MISC',
  'score': 0.87388104,
  'index': 60,
  'word': 'Viking',
  'start': 240,
  'end': 246},
 {'entity': 'I-MISC',
  'score': 0.9686202,
  'index': 61,
  'word': '1',
  'start': 247,
  'end': 248},
 {'entity': 'I-MISC',
  'score': 0.6234842,
  'index': 98,
  'word': 'Mart',
  'start': 407,
  'end': 411},
 {'entity': 'I-MISC',
  'score': 0.7816169,
  'index': 99,
  'word': '##ian',
  'start': 411,
  'end': 414},
 {'entity': 'B-LOC',
  'score': 0.91

In [83]:

with open("40 DunnBC22bert-base-multilingual-cased-fine_tuned-ner-WikiNeural_Multilingual.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC      8
B-ORG      3
B-PER      2
I-LOC      3
I-MISC    15
I-PER      3
dtype: int64


entity  word    
B-LOC   American    1
        C           1
        Egypt       1
        Mars        5
B-ORG   NASA        3
B-PER   Mali        1
        Michael     1
I-LOC   ##onia      1
        ##yd        1
        West        1
I-MISC  ##ara       1
        ##biter     1
        ##ian       1
        ##ion       1
        ##oh        1
        1           1
        Face        1
        Mars        2
        Mart        1
        On          1
        Or          1
        Ph          1
        Viking      2
I-PER   ##n         2
        Mali        1
dtype: int64

## 41 rollerhafeezh-amikom/xlm-roberta-base-ner-silvanus 

In [85]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("rollerhafeezh-amikom/xlm-roberta-base-ner-silvanus")
model = AutoModelForTokenClassification.from_pretrained("rollerhafeezh-amikom/xlm-roberta-base-ner-silvanus")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'B-LOC',
  'score': 0.9711031,
  'index': 103,
  'word': '▁Cy',
  'start': 435,
  'end': 437},
 {'entity': 'I-LOC',
  'score': 0.97973377,
  'index': 104,
  'word': 'do',
  'start': 437,
  'end': 439},
 {'entity': 'I-LOC',
  'score': 0.98118365,
  'index': 105,
  'word': 'nia',
  'start': 439,
  'end': 442},
 {'entity': 'B-DAT',
  'score': 0.99709177,
  'index': 315,
  'word': '▁April',
  'start': 1297,
  'end': 1302},
 {'entity': 'I-DAT',
  'score': 0.9621371,
  'index': 316,
  'word': '▁5',
  'start': 1303,
  'end': 1304},
 {'entity': 'I-DAT',
  'score': 0.9620826,
  'index': 317,
  'word': ',',
  'start': 1304,
  'end': 1305},
 {'entity': 'I-DAT',
  'score': 0.98092973,
  'index': 318,
  'word': '▁1998.',
  'start': 1306,
  'end': 1311},
 {'entity': 'B-DAT',
  'score': 0.9965664,
  'index': 395,
  'word': '▁April',
  'start': 1652,
  'end': 1657},
 {'entity': 'I-DAT',
  'score': 0.9623601,
  'index': 396,
  'word': '▁8',
  'start': 1658,
  'end': 1659},
 {'entity': 'I-DA

In [86]:
with open("41 rollerhafeezh-amikomxlm-roberta-base-ner-silvanus.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-DAT    2
B-LOC    2
I-DAT    6
I-LOC    3
dtype: int64


entity  word     
B-DAT   ▁April       2
B-LOC   ▁American    1
        ▁Cy          1
I-DAT   ,            2
        ▁1998.       1
        ▁2001        1
        ▁5           1
        ▁8           1
I-LOC   do           1
        nia          1
        ▁West        1
dtype: int64

## 42 orgcatorg/distilbert-base-multilingual-cased-ner

In [88]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("orgcatorg/distilbert-base-multilingual-cased-ner")
model = AutoModelForTokenClassification.from_pretrained("orgcatorg/distilbert-base-multilingual-cased-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-LOC',
  'score': 0.9980641,
  'index': 104,
  'word': 'C',
  'start': 435,
  'end': 436},
 {'entity': 'B-LOC',
  'score': 0.99705327,
  'index': 105,
  'word': '##yd',
  'start': 436,
  'end': 438},
 {'entity': 'B-LOC',
  'score': 0.9987865,
  'index': 106,
  'word': '##onia',
  'start': 438,
  'end': 442},
 {'entity': 'B-PER',
  'score': 0.66370535,
  'index': 121,
  'word': 'Egypt',
  'start': 496,
  'end': 501},
 {'entity': 'B-PER',
  'score': 0.6953692,
  'index': 122,
  'word': '##ion',
  'start': 501,
  'end': 504},
 {'entity': 'I-PER',
  'score': 0.6620473,
  'index': 123,
  'word': 'Ph',
  'start': 505,
  'end': 507},
 {'entity': 'I-PER',
  'score': 0.7284523,
  'index': 124,
  'word': '##ara',
  'start': 507,
  'end': 510},
 {'entity': 'I-PER',
  'score': 0.704528,
  'index': 125,
  'word': '##oh',
  'start': 510,
  'end': 512},
 {'entity': 'B-PER',
  'score': 0.9904669,
  'index': 316,
  'word': 'Michael',
  'start': 1312,
  'end': 1319},
 {'entity': 'I-PER',
 

In [89]:
with open("42 orgcatorgdistilbert-base-multilingual-cased-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC    4
B-PER    3
I-LOC    1
I-PER    5
dtype: int64


entity  word    
B-LOC   ##onia      1
        ##yd        1
        American    1
        C           1
B-PER   ##ion       1
        Egypt       1
        Michael     1
I-LOC   West        1
I-PER   ##ara       1
        ##n         1
        ##oh        1
        Mali        1
        Ph          1
dtype: int64

## 43 orgcatorg/xlm-v-base-ner

In [92]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("orgcatorg/xlm-v-base-ner")
model = AutoModelForTokenClassification.from_pretrained("orgcatorg/xlm-v-base-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-ORG',
  'score': 0.8529484,
  'index': 8,
  'word': '▁NASA',
  'start': 15,
  'end': 20},
 {'entity': 'B-ORG',
  'score': 0.67219996,
  'index': 23,
  'word': '▁Face',
  'start': 87,
  'end': 92},
 {'entity': 'I-ORG',
  'score': 0.9593816,
  'index': 24,
  'word': '▁On',
  'start': 92,
  'end': 95},
 {'entity': 'I-ORG',
  'score': 0.98047435,
  'index': 25,
  'word': '▁Mars',
  'start': 95,
  'end': 100},
 {'entity': 'B-ORG',
  'score': 0.97809476,
  'index': 57,
  'word': '▁Viking',
  'start': 239,
  'end': 246},
 {'entity': 'I-ORG',
  'score': 0.97545755,
  'index': 58,
  'word': '▁1',
  'start': 246,
  'end': 248},
 {'entity': 'I-ORG',
  'score': 0.5685065,
  'index': 59,
  'word': '▁space',
  'start': 248,
  'end': 254},
 {'entity': 'B-LOC',
  'score': 0.7317715,
  'index': 93,
  'word': '▁Marti',
  'start': 406,
  'end': 412},
 {'entity': 'B-LOC',
  'score': 0.77504426,
  'index': 94,
  'word': 'an',
  'start': 412,
  'end': 414},
 {'entity': 'B-LOC',
  'score': 0.7

In [93]:
with open("43 orgcatorgxlm-v-base-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-LOC    8
B-ORG    4
B-PER    1
I-LOC    2
I-ORG    6
I-PER    1
dtype: int64


entity  word     
B-LOC   an           1
        on           1
        te           1
        ▁American    1
        ▁Cy          1
        ▁Egypti      1
        ▁Marti       1
        ▁but         1
B-ORG   ▁Face        1
        ▁Mars        1
        ▁NASA        1
        ▁Viking      1
B-PER   ▁Michael     1
I-LOC   h            1
        ▁Pharao      1
I-ORG   er           1
        ▁1           1
        ▁Mars        1
        ▁On          1
        ▁Orbit       1
        ▁space       1
I-PER   ▁Malin       1
dtype: int64

## 44 dejanseo/LinkBERT-XL

In [95]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("dejanseo/LinkBERT-XL")
model = AutoModelForTokenClassification.from_pretrained("dejanseo/LinkBERT-XL")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'LABEL_0',
  'score': 0.99891233,
  'index': 1,
  'word': '▁So',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_0',
  'score': 0.99892116,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'LABEL_0',
  'score': 0.9979603,
  'index': 3,
  'word': '▁if',
  'start': 4,
  'end': 6},
 {'entity': 'LABEL_0',
  'score': 0.99819654,
  'index': 4,
  'word': '▁you',
  'start': 7,
  'end': 10},
 {'entity': 'LABEL_0',
  'score': 0.9984597,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'LABEL_0',
  'score': 0.99839216,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'LABEL_0',
  'score': 0.9984572,
  'index': 7,
  'word': '▁a',
  'start': 14,
  'end': 15},
 {'entity': 'LABEL_0',
  'score': 0.9959991,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': 'LABEL_0',
  'score': 0.99812514,
  'index': 9,
  'word': '▁scientist',
  'start': 21,
  'end': 30},
 {'entity': 'LABEL_0',
  'score': 0.9979564,
  'index

## 45 HiTZ/mbert-argmining-abstrct-en-es

In [97]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("HiTZ/mbert-argmining-abstrct-en-es")
model = AutoModelForTokenClassification.from_pretrained("HiTZ/mbert-argmining-abstrct-en-es")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[]

## 46 HiTZ/mdeberta-expl-extraction-multi 

In [99]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("HiTZ/mdeberta-expl-extraction-multi")
model = AutoModelForTokenClassification.from_pretrained("HiTZ/mdeberta-expl-extraction-multi")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

C:\Users\NW\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at HiTZ/mdeberta-expl-extraction-multi and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'LABEL_1',
  'score': 0.69964296,
  'index': 1,
  'word': '▁So',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_1',
  'score': 0.68438363,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'LABEL_1',
  'score': 0.76636976,
  'index': 3,
  'word': '▁if',
  'start': 3,
  'end': 6},
 {'entity': 'LABEL_1',
  'score': 0.7428422,
  'index': 4,
  'word': '▁you',
  'start': 6,
  'end': 10},
 {'entity': 'LABEL_1',
  'score': 0.72554,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'LABEL_1',
  'score': 0.73893636,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'LABEL_1',
  'score': 0.73846024,
  'index': 7,
  'word': '▁',
  'start': 13,
  'end': 14},
 {'entity': 'LABEL_1',
  'score': 0.7346474,
  'index': 8,
  'word': 'a',
  'start': 14,
  'end': 15},
 {'entity': 'LABEL_1',
  'score': 0.71560675,
  'index': 9,
  'word': '▁NASA',
  'start': 15,
  'end': 20},
 {'entity': 'LABEL_1',
  'score': 0.7557671,
  'index': 10,
  'w

## 47 rollerhafeezh-amikom/xlm-roberta-base-ner-augmentation

In [101]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("rollerhafeezh-amikom/xlm-roberta-base-ner-augmentation")
model = AutoModelForTokenClassification.from_pretrained("rollerhafeezh-amikom/xlm-roberta-base-ner-augmentation")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'B-LOC',
  'score': 0.9935009,
  'index': 103,
  'word': '▁Cy',
  'start': 435,
  'end': 437},
 {'entity': 'I-LOC',
  'score': 0.9510317,
  'index': 104,
  'word': 'do',
  'start': 437,
  'end': 439},
 {'entity': 'I-LOC',
  'score': 0.96693367,
  'index': 105,
  'word': 'nia',
  'start': 439,
  'end': 442},
 {'entity': 'B-LOC',
  'score': 0.7514709,
  'index': 264,
  'word': '▁Mars',
  'start': 1088,
  'end': 1092},
 {'entity': 'B-DAT',
  'score': 0.9943066,
  'index': 315,
  'word': '▁April',
  'start': 1297,
  'end': 1302},
 {'entity': 'I-DAT',
  'score': 0.99545693,
  'index': 316,
  'word': '▁5',
  'start': 1303,
  'end': 1304},
 {'entity': 'I-DAT',
  'score': 0.9937779,
  'index': 317,
  'word': ',',
  'start': 1304,
  'end': 1305},
 {'entity': 'I-DAT',
  'score': 0.99656725,
  'index': 318,
  'word': '▁1998.',
  'start': 1306,
  'end': 1311},
 {'entity': 'B-DAT',
  'score': 0.9939989,
  'index': 395,
  'word': '▁April',
  'start': 1652,
  'end': 1657},
 {'entity': 'I-

In [102]:


with open("47 rollerhafeezh-amikomxlm-roberta-base-ner-augmentation.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-DAT    2
B-LOC    3
I-DAT    6
I-LOC    3
dtype: int64


entity  word     
B-DAT   ▁April       2
B-LOC   ▁American    1
        ▁Cy          1
        ▁Mars        1
I-DAT   ,            2
        ▁1998.       1
        ▁2001        1
        ▁5           1
        ▁8           1
I-LOC   do           1
        nia          1
        ▁West        1
dtype: int64

## 48 brettlin/distilbert-base-uncased-finetuned-ner

In [104]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("brettlin/distilbert-base-uncased-finetuned-ner")
model = AutoModelForTokenClassification.from_pretrained("brettlin/distilbert-base-uncased-finetuned-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'LABEL_0',
  'score': 0.9996531,
  'index': 1,
  'word': 'so',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_0',
  'score': 0.9995907,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'LABEL_0',
  'score': 0.99943215,
  'index': 3,
  'word': 'if',
  'start': 4,
  'end': 6},
 {'entity': 'LABEL_0',
  'score': 0.9986737,
  'index': 4,
  'word': 'you',
  'start': 7,
  'end': 10},
 {'entity': 'LABEL_0',
  'score': 0.9979754,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'LABEL_0',
  'score': 0.995782,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'LABEL_0',
  'score': 0.98806703,
  'index': 7,
  'word': 'a',
  'start': 14,
  'end': 15},
 {'entity': 'LABEL_0',
  'score': 0.60142565,
  'index': 8,
  'word': 'nasa',
  'start': 16,
  'end': 20},
 {'entity': 'LABEL_75',
  'score': 0.74799323,
  'index': 9,
  'word': 'scientist',
  'start': 21,
  'end': 30},
 {'entity': 'LABEL_0',
  'score': 0.99963987,
  'index': 10,

In [105]:
with open("48 brettlindistilbert-base-uncased-finetuned-ner.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()


# ['O', 	0
# 'B-O', 	1
# 'I-O', 	2
# 'L-O', 	3
# 'U-O', 	4
# 'B-PER', 	5
# 'I-PER', 	6
# 'L-PER', 	7
# 'U-PER', 	8
# 'B-LOC', 	9
# 'I-LOC', 	10
# 'L-LOC', 	11
# 'U-LOC', 	12
# 'B-ORG', 	13
# 'I-ORG', 	14
# 'L-ORG', 	15
# 'U-ORG', 	16
# 'B-NRP', 	17
# 'I-NRP', 	18
# 'L-NRP', 	19
# 'U-NRP', 	20
# 'B-DATE_TIME', 	21
# 'I-DATE_TIME', 	22
# 'L-DATE_TIME', 	23
# 'U-DATE_TIME', 	24
# 'B-CREDIT_CARD', 	25
# 'I-CREDIT_CARD', 	26
# 'L-CREDIT_CARD', 	27
# 'U-CREDIT_CARD', 	28
# 'B-URL', 	29
# 'I-URL', 	30
# 'L-URL', 	31
# 'U-URL', 	32
# 'B-IBAN_CODE', 	33
# 'I-IBAN_CODE', 	34
# 'L-IBAN_CODE', 	35
# 'U-IBAN_CODE', 	36
# 'B-US_BANK_NUMBER', 	37
# 'I-US_BANK_NUMBER', 	38
# 'L-US_BANK_NUMBER', 	39
# 'U-US_BANK_NUMBER', 	40
# 'B-PHONE_NUMBER', 	41
# 'I-PHONE_NUMBER', 	42
# 'L-PHONE_NUMBER', 	43
# 'U-PHONE_NUMBER', 	44
# 'B-US_SSN', 	45
# 'I-US_SSN', 	46
# 'L-US_SSN', 	47
# 'U-US_SSN', 	48
# 'B-US_PASSPORT', 	49
# 'I-US_PASSPORT', 	50
# 'L-US_PASSPORT', 	51
# 'U-US_PASSPORT', 	52
# 'B-US_DRIVER_LICENSE', 	53
# 'I-US_DRIVER_LICENSE', 	54
# 'L-US_DRIVER_LICENSE', 	55
# 'U-US_DRIVER_LICENSE', 	56
# 'B-US_LICENSE_PLATE', 	57
# 'I-US_LICENSE_PLATE', 	58
# 'L-US_LICENSE_PLATE', 	59
# 'U-US_LICENSE_PLATE', 	60
# 'B-IP_ADDRESS', 	61
# 'I-IP_ADDRESS', 	62
# 'L-IP_ADDRESS', 	63
# 'U-IP_ADDRESS', 	64
# 'B-US_ITIN', 	65
# 'I-US_ITIN', 	66
# 'L-US_ITIN', 	67
# 'U-US_ITIN', 	68
# 'B-EMAIL_ADDRESS', 	69
# 'I-EMAIL_ADDRESS', 	70
# 'L-EMAIL_ADDRESS', 	71
# 'U-EMAIL_ADDRESS', 	72
# 'B-TITLE', 	73
# 'I-TITLE', 	74
# 'L-TITLE', 	75
# 'U-TITLE', 	76
# 'B-COORDINATE', 	77
# 'I-COORDINATE', 	78
# 'L-COORDINATE', 	79
# 'U-COORDINATE', 	80
# 'B-IMEI', 	81
# 'I-IMEI', 	82
# 'L-IMEI', 	83
# 'U-IMEI', 	84
# 'B-PASSWORD', 	85
# 'I-PASSWORD', 	86
# 'L-PASSWORD', 	87
# 'U-PASSWORD', 	88
# 'B-LICENSE_PLATE', 	89
# 'I-LICENSE_PLATE', 	90
# 'L-LICENSE_PLATE', 	91
# 'U-LICENSE_PLATE', 	92
# 'B-CURRENCY', 	93
# 'I-CURRENCY', 	94
# 'L-CURRENCY', 	95
# 'U-CURRENCY', 	96
# 'B-FINANCIAL', 	97
# 'I-FINANCIAL', 	98
# 'L-FINANCIAL', 	99
# 'U-FINANCIAL', 	100
# 'B-ROUTING_NUMBER', 	101
# 'I-ROUTING_NUMBER', 	102
# 'L-ROUTING_NUMBER', 	103
# 'U-ROUTING_NUMBER', 	104
# 'B-SWIFT_CODE', 	105
# 'I-SWIFT_CODE', 	106
# 'L-SWIFT_CODE', 	107
# 'U-SWIFT_CODE', 	108
# 'B-MAC_ADDRESS', 	109
# 'I-MAC_ADDRESS', 	110
# 'L-MAC_ADDRESS', 	111
# 'U-MAC_ADDRESS', 	112
# 'B-AGE', 	113
# 'I-AGE', 	114
# 'L-AGE', 	115
# 'U-AGE']	116



entity
LABEL_0     423
LABEL_10      4
LABEL_11      1
LABEL_14      9
LABEL_20      3
LABEL_24      7
LABEL_5       1
LABEL_7       2
LABEL_73      2
LABEL_74      4
LABEL_75      4
dtype: int64


entity    word        
LABEL_0                    9
          ##ct             1
          ##ern            1
          ##form           2
          ##forms          1
          ##less           1
          ##n              1
          ##r              1
          ##rra            1
          ,               33
          -                2
          .               12
          25               1
          3                1
          ;                1
          ?                1
          a               10
          able             1
          about            1
          absolute         1
          ago              1
          alien            2
          aliens           1
          all              3
          amazing          1
          an               2
          ancient          1
          and              8
          announced        1
          another          3
          any              1
          appeared         1
          are              1
          around    

## 49 papluca/xlm-roberta-base-language-detection

In [117]:

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
model = AutoModelForTokenClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")
nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'entity': 'en',
  'score': 0.11591221,
  'index': 1,
  'word': '▁So',
  'start': 0,
  'end': 2},
 {'entity': 'en',
  'score': 0.124623634,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'el',
  'score': 0.12547147,
  'index': 3,
  'word': '▁if',
  'start': 4,
  'end': 6},
 {'entity': 'en',
  'score': 0.13911082,
  'index': 4,
  'word': '▁you',
  'start': 7,
  'end': 10},
 {'entity': 'en',
  'score': 0.17602253,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'el',
  'score': 0.11501573,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'en',
  'score': 0.14821951,
  'index': 7,
  'word': '▁a',
  'start': 14,
  'end': 15},
 {'entity': 'el',
  'score': 0.10118,
  'index': 8,
  'word': '▁NASA',
  'start': 16,
  'end': 20},
 {'entity': 'el',
  'score': 0.09721276,
  'index': 9,
  'word': '▁scientist',
  'start': 21,
  'end': 30},
 {'entity': 'el',
  'score': 0.12020393,
  'index': 10,
  'word': ',',
  'start': 30,
  'end': 3

In [125]:

with open("49 paplucaxlm-roberta-base-language-detection.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

aux.groupby(['entity']) \
       .agg({'entity':'size', 'score':'mean'}) \
       #.rename(columns={'text':'count','sent':'mean_sent'}) \
       #.reset_index()

entity
ar      8
el    305
en    178
ja      1
pl      1
dtype: int64


,entity,score
entity,,
ar,8,0.084216
el,305,0.120293
en,178,0.143087
ja,1,0.077624
pl,1,0.075424


## 50 mbruton/spa_en_mBERT

In [109]:

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("mbruton/spa_en_mBERT")
model = AutoModelForTokenClassification.from_pretrained("mbruton/spa_en_mBERT")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results



[{'entity': 'r0:arg1|tem',
  'score': 0.29541913,
  'index': 4,
  'word': 'you',
  'start': 7,
  'end': 10},
 {'entity': 'r0:root',
  'score': 0.4976404,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'r0:root',
  'score': 0.9618234,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'r1:arg1|tem',
  'score': 0.7678349,
  'index': 11,
  'word': 'you',
  'start': 32,
  'end': 35},
 {'entity': 'r1:root',
  'score': 0.6855395,
  'index': 13,
  'word': 'be',
  'start': 43,
  'end': 45},
 {'entity': 'r1:arg2|atr',
  'score': 0.93614143,
  'index': 14,
  'word': 'able',
  'start': 46,
  'end': 50},
 {'entity': 'r2:arg2|ben',
  'score': 0.5274996,
  'index': 17,
  'word': 'me',
  'start': 59,
  'end': 61},
 {'entity': 'r2:arg1|pat',
  'score': 0.36818993,
  'index': 20,
  'word': 'story',
  'start': 72,
  'end': 77},
 {'entity': 'r4:arg1|tem',
  'score': 0.4968183,
  'index': 27,
  'word': 'which',
  'start': 102,
  'end': 107},
 {'entity': 'r3:root'

In [110]:

with open("50 mbrutonspa_en_mBERT.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
r0:arg1|tem      1
r0:root          2
r10:root       113
r1:arg1|tem      1
r1:arg2|atr      1
r1:root          1
r2:arg1|pat      1
r2:arg2|ben      1
r3:root          1
r4:arg0|agt      1
r4:arg1|tem      2
r4:arg2|atr      2
r5:arg0|agt      1
r5:arg1|tem      2
r5:root          1
r6:arg1|tem      1
r6:root          1
r7:arg0|agt      4
r7:arg1|pat      2
r7:arg1|tem      2
r7:argM|tmp      1
r7:root          3
r8:arg0|agt      3
r8:arg1|pat      3
r8:arg1|tem     29
r8:arg2|atr      7
r8:argM|adv      7
r8:root          2
r9:root          9
dtype: int64


entity       word      
r0:arg1|tem  you           1
r0:root                    1
             re            1
r10:root     ##adow        1
             ##ct          1
             ##d           2
             ##emble       1
             ##er          1
             ##ern         1
             ##form        1
             ##ing         1
             ##n           2
             ##rra         1
             ##s           3
             ##te          1
             Mali          1
             What          1
             With          1
             alien         1
             announced     1
             are           1
             around        1
             at            1
             be            1
             became        1
             but           1
             by            1
             can           1
             captured      1
             common        1
             could         1
             day           1
             decided       1
             did   

## 51 BSC-LT/roberta_model_for_anonimization

In [132]:
text="""Entonces, si eres un científico de la NASA, deberías poder contarme toda la historia sobre la Cara en Marte, que obviamente es evidencia de que hay vida en Marte y que la cara fue creada por extraterrestres, ¿correcto?" No. Hace veinticinco años, nuestra nave espacial Viking 1 estaba dando vueltas alrededor del planeta, tomando fotografías, cuando detectó la sombra de un rostro humano. Los científicos estadounidenses descubrieron que se trataba simplemente de otra mesa marciana, común alrededor de Cydonia, solo que ésta tenía sombras. Eso lo hizo parecer un faraón egipcio. Muy pocos días después, revelamos la imagen para que todos la vieran, y nos aseguramos de notar que era una enorme formación rocosa que simplemente se parecía a una cabeza y un rostro humanos, pero todo era. formado por sombras Sólo lo anunciamos porque pensamos que sería una buena manera de involucrar al público con los hallazgos de la NASA y atraer la atención a Marte, y así fue.

El rostro de Marte pronto se convirtió en un ícono pop; filmada en películas, apareció en libros, revistas, programas de radio y en las colas de las cajas de las tiendas de comestibles durante 25 años. Algunas personas pensaron que la forma natural del relieve era evidencia de vida en Marte, y que los científicos queríamos ocultarla, pero en realidad, los defensores del presupuesto de la NASA desearían que hubiera una civilización antigua en Marte. Decidimos tomar otra foto solo para asegurarnos de no estar equivocados, el 5 de abril de 1998. Michael Malin y su equipo de cámara de Mars Orbiter tomaron una fotografía que era diez veces más nítida que las fotografías originales del Viking, revelando una forma de relieve natural, que No significaba ningún monumento alienígena. "Pero esa imagen no era muy clara en absoluto, lo que podría significar que las marcas alienígenas estaban ocultas por la neblina" Bueno, no, sí, ese rumor comenzó, pero para demostrar que estaban equivocados, el 8 de abril de 2001 decidimos tomar otra fotografía, asegurándonos de que Era un día de verano sin nubes. El equipo de Malin capturó una fotografía asombrosa utilizando la revolución máxima absoluta de la cámara. Con esta cámara puedes discernir cosas en una imagen digital, 3 veces más grande que el tamaño de un píxel, lo que significa que si hubiera señales de vida, podrías ver fácilmente cuáles eran. Lo que la imagen mostraba era la colina o mesa, que son accidentes geográficos comunes en el oeste americano."""

tokenizer = AutoTokenizer.from_pretrained("BSC-LT/roberta_model_for_anonimization")
model = AutoModelForTokenClassification.from_pretrained("BSC-LT/roberta_model_for_anonimization")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results

[{'entity': 'S-ORG',
  'score': 0.9931144,
  'index': 10,
  'word': 'ĠNASA',
  'start': 38,
  'end': 42},
 {'entity': 'S-OTH',
  'score': 0.9904603,
  'index': 21,
  'word': 'ĠCara',
  'start': 94,
  'end': 98},
 {'entity': 'S-LOC',
  'score': 0.9779458,
  'index': 23,
  'word': 'ĠMarte',
  'start': 102,
  'end': 107},
 {'entity': 'S-LOC',
  'score': 0.9959857,
  'index': 34,
  'word': 'ĠMarte',
  'start': 156,
  'end': 161},
 {'entity': 'B-OTH',
  'score': 0.9917984,
  'index': 57,
  'word': 'ĠVik',
  'start': 269,
  'end': 272},
 {'entity': 'I-OTH',
  'score': 0.9757009,
  'index': 58,
  'word': 'ing',
  'start': 272,
  'end': 275},
 {'entity': 'E-OTH',
  'score': 0.9817541,
  'index': 59,
  'word': 'Ġ1',
  'start': 276,
  'end': 277},
 {'entity': 'S-LOC',
  'score': 0.99622846,
  'index': 96,
  'word': 'ĠC',
  'start': 503,
  'end': 504},
 {'entity': 'S-ORG',
  'score': 0.990908,
  'index': 182,
  'word': 'ĠNASA',
  'start': 919,
  'end': 923},
 {'entity': 'S-LOC',
  'score': 0.9959

In [133]:

with open("51 BSC-LTroberta_model_for_anonimization.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
B-ORG    1
B-OTH    1
B-PER    1
E-ORG    3
E-OTH    1
E-PER    2
I-OTH    1
S-LOC    7
S-ORG    3
S-OTH    2
S-PER    1
dtype: int64


entity  word    
B-ORG   ĠMars       1
B-OTH   ĠVik        1
B-PER   ĠMichael    1
E-ORG   bi          1
        ter         1
        ĠOr         1
E-OTH   Ġ1          1
E-PER   in          1
        ĠMal        1
I-OTH   ing         1
S-LOC   ĠC          1
        ĠMarte      6
S-ORG   ĠNASA       3
S-OTH   ĠCara       1
        ĠVik        1
S-PER   ĠMal        1
dtype: int64

## 52 aymurai/anonymizer-beto-cased-flair

In [111]:


from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("aymurai/anonymizer-beto-cased-flair")
model = AutoModelForTokenClassification.from_pretrained("aymurai/anonymizer-beto-cased-flair")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results



C:\Users\NW\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: aymurai/anonymizer-beto-cased-flair does not appear to have a file named config.json. Checkout 'https://huggingface.co/aymurai/anonymizer-beto-cased-flair/tree/main' for available files.

## 53 google-bert/bert-large-cased-whole-word-masking

In [113]:


from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-cased-whole-word-masking")
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-large-cased-whole-word-masking")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results



Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-large-cased-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'entity': 'LABEL_1',
  'score': 0.7333207,
  'index': 1,
  'word': 'So',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_1',
  'score': 0.7897263,
  'index': 2,
  'word': ',',
  'start': 2,
  'end': 3},
 {'entity': 'LABEL_1',
  'score': 0.61105645,
  'index': 3,
  'word': 'if',
  'start': 4,
  'end': 6},
 {'entity': 'LABEL_1',
  'score': 0.6179384,
  'index': 4,
  'word': 'you',
  'start': 7,
  'end': 10},
 {'entity': 'LABEL_1',
  'score': 0.51219755,
  'index': 5,
  'word': "'",
  'start': 10,
  'end': 11},
 {'entity': 'LABEL_1',
  'score': 0.7380822,
  'index': 6,
  'word': 're',
  'start': 11,
  'end': 13},
 {'entity': 'LABEL_1',
  'score': 0.65777117,
  'index': 7,
  'word': 'a',
  'start': 14,
  'end': 15},
 {'entity': 'LABEL_0',
  'score': 0.53536695,
  'index': 8,
  'word': 'NASA',
  'start': 16,
  'end': 20},
 {'entity': 'LABEL_1',
  'score': 0.5658676,
  'index': 9,
  'word': 'scientist',
  'start': 21,
  'end': 30},
 {'entity': 'LABEL_1',
  'score': 0.7322473,
  'index': 10,


## 54 PlanTL-GOB-ES/es_anonimization_core_lg

In [114]:


from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/es_anonimization_core_lg")
model = AutoModelForTokenClassification.from_pretrained("PlanTL-GOB-ES/es_anonimization_core_lg")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
ner_results



C:\Users\NW\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: PlanTL-GOB-ES/es_anonimization_core_lg does not appear to have a file named config.json. Checkout 'https://huggingface.co/PlanTL-GOB-ES/es_anonimization_core_lg/tree/main' for available files.

# distilbert-base-cased

In [14]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])
model_args={"trust_remote_code": True}
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-cased", **model_args)
with torch.no_grad():
    logits = model(input_ids).logits
predicted_token_class_ids = logits.argmax(-1)

predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

labels = predicted_token_class_ids
loss = model(input_ids, labels=labels).loss
print('loss:',round(loss.item(), 2))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loss: 0.47


In [15]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

In [21]:
import json
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("distilbert/distilbert-base-cased")
classifier = pipeline("token-classification", model=model, tokenizer=tokenizer)
salida=classifier(text)
print(salida)



model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'entity': 'LABEL_1', 'score': 0.52068406, 'index': 1, 'word': 'So', 'start': 0, 'end': 2}, {'entity': 'LABEL_1', 'score': 0.52839065, 'index': 2, 'word': ',', 'start': 2, 'end': 3}, {'entity': 'LABEL_1', 'score': 0.56141883, 'index': 3, 'word': 'if', 'start': 4, 'end': 6}, {'entity': 'LABEL_0', 'score': 0.5763298, 'index': 4, 'word': 'you', 'start': 7, 'end': 10}, {'entity': 'LABEL_0', 'score': 0.5900621, 'index': 5, 'word': "'", 'start': 10, 'end': 11}, {'entity': 'LABEL_0', 'score': 0.5865794, 'index': 6, 'word': 're', 'start': 11, 'end': 13}, {'entity': 'LABEL_0', 'score': 0.54452276, 'index': 7, 'word': 'a', 'start': 14, 'end': 15}, {'entity': 'LABEL_0', 'score': 0.56138283, 'index': 8, 'word': 'NASA', 'start': 16, 'end': 20}, {'entity': 'LABEL_0', 'score': 0.5386717, 'index': 9, 'word': 'scientist', 'start': 21, 'end': 30}, {'entity': 'LABEL_1', 'score': 0.5386055, 'index': 10, 'word': ',', 'start': 30, 'end': 31}, {'entity': 'LABEL_0', 'score': 0.5242754, 'index': 11, 'word': '

In [22]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

In [8]:
with open("distilbert-base-cased.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity']).size())
aux.groupby(['entity', 'word']).size()

entity
LABEL_0    145
LABEL_1    323
dtype: int64


entity   word  
LABEL_0  ##er      1
         ##n       1
         ##ness    1
         ##out     1
         ##sa      1
                  ..
LABEL_1  with      1
         wrong     1
         years     2
         yes       1
         you       4
Length: 275, dtype: int64

# distilbert-tuned-4labels

In [1]:
from transformers import pipeline
checkpoint = "distilbert-tuned-4labels"
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

text="""So, if you're a NASA scientist, you should be able to tell me the whole story about the Face On Mars, which obviously is evidence that there is life on Mars, and that the face was created by aliens, correct?" No, twenty five years ago, our Viking 1 spacecraft was circling the planet, snapping photos, when it spotted the shadowy likeness of a human face. Us scientists figured out that it was just another Martian mesa, common around Cydonia, only this one had shadows that made it look like an Egyption Pharaoh. Very few days later, we revealed the image for all to see, and we made sure to note that it was a huge rock formation that just resembled a human head and face, but all of it was formed by shadows. We only announced it because we thought it would be a good way to engage the public with NASA's findings, and atrract attention to Mars-- and it did.

The face on Mars soon became a pop icon; shot in movies, appeared in books, magazines, radio talk shows, and haunted grocery store checkout lines for 25 years. Some people thought the natural landform was evidence of life on Mars, and that us scientists wanted to hide it, but really, the defenders of the NASA budget wish there was ancient civilization on Mars. We decided to take another shot just to make sure we weren't wrong, on April 5, 1998. Michael Malin and his Mars Orbiter camera team took a picture that was ten times sharper than the original Viking photos, revealing a natural landform, which meant no alien monument. "But that picture wasn't very clear at all, which could mean alien markings were hidden by haze" Well no, yes that rumor started, but to prove them wrong on April 8, 2001 we decided to take another picture, making sure it was a cloudless summer day. Malin's team captured an amazing photo using the camera's absolute maximum revolution. With this camera you can discern things in a digital image, 3 times bigger than the pixel size which means if there were any signs of life, you could easily see what they were. What the picture showed was the butte or mesa, which are landforms common around the American West."""

token_classifier(text)



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'ORG',
  'score': 0.9631609,
  'word': 'NASA',
  'start': 16,
  'end': 20},
 {'entity_group': 'MISC',
  'score': 0.9983225,
  'word': 'Face On Mars',
  'start': 88,
  'end': 100},
 {'entity_group': 'LOC',
  'score': 0.81088793,
  'word': 'Mars',
  'start': 152,
  'end': 156},
 {'entity_group': 'MISC',
  'score': 0.9847041,
  'word': 'Viking 1',
  'start': 240,
  'end': 248},
 {'entity_group': 'MISC',
  'score': 0.9998406,
  'word': 'Martian',
  'start': 407,
  'end': 414},
 {'entity_group': 'LOC',
  'score': 0.7726369,
  'word': 'Cydonia',
  'start': 435,
  'end': 442},
 {'entity_group': 'MISC',
  'score': 0.8863019,
  'word': 'Egyption Phara',
  'start': 496,
  'end': 510},
 {'entity_group': 'PER',
  'score': 0.7494665,
  'word': '##oh',
  'start': 510,
  'end': 512},
 {'entity_group': 'ORG',
  'score': 0.88716674,
  'word': 'NASA',
  'start': 801,
  'end': 805},
 {'entity_group': 'LOC',
  'score': 0.9616925,
  'word': 'Mars',
  'start': 843,
  'end': 847},
 {'entity

In [5]:

import json
with open("json_entrenado.json", encoding='utf-8') as f:
    data = json.load(f)
aux=obtener_dataframe(data)
aux
print(aux.groupby(['entity_group']).size())
aux.groupby(['entity_group', 'word']).size()

entity_group
DATE    4
LOC     6
MISC    9
ORG     3
PER     2
dtype: int64


entity_group  word          
DATE          1998              1
              2001              1
              April 5           1
              April 8           1
LOC           Cydonia           1
              Mars              5
MISC          American          1
              Egyption Phara    1
              Face On Mars      1
              Malin             1
              Mars Orbiter      1
              Martian           1
              Viking            1
              Viking 1          1
              West              1
ORG           NASA              3
PER           ##oh              1
              Michael Malin     1
dtype: int64